### Identifier les valeurs aberrantes dans `products.csv`

Utiliser `pandas` pour charger et analyser les données du fichier CSV `products.csv`. Repérer les valeurs aberrantes (ordre de grandeur : quelques centaines).

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
DATA_DIR = Path("../data")
product_file_path = Path(DATA_DIR, "products.csv")

In [3]:
product_df = pd.read_csv(product_file_path, low_memory=False)

In [4]:
print(product_df)

        Unnamed: 0           code  fat_100g  saturated-fat_100g  sugars_100g  \
0                0           3087       NaN                 NaN          NaN   
1                1           4530     28.57               28.57        14.29   
2                2           4559     17.86                0.00        17.86   
3                3          16087     57.14                5.36         3.57   
4                4          16094      1.43                 NaN          NaN   
...            ...            ...       ...                 ...          ...   
320767      320767  9948282780603       NaN                 NaN          NaN   
320768      320768       99567453      0.00                0.00         0.00   
320769      320769  9970229501521       NaN                 NaN          NaN   
320770      320770  9980282863788       NaN                 NaN          NaN   
320771      320771   999990026839      0.00                 NaN         0.00   

        fiber_100g  proteins_100g  salt

In [5]:
product_df.head()

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
0,0,3087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000
1,1,4530,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000
2,2,4559,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500
3,3,16087,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372
4,4,16094,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000


In [6]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          320772 non-null  int64  
 1   code                320749 non-null  object 
 2   fat_100g            243891 non-null  float64
 3   saturated-fat_100g  229554 non-null  float64
 4   sugars_100g         244971 non-null  float64
 5   fiber_100g          200886 non-null  float64
 6   proteins_100g       259922 non-null  float64
 7   salt_100g           255510 non-null  float64
 8   sodium_100g         255463 non-null  float64
 9   autre               320772 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 24.5+ MB


In [7]:
product_df.describe(include="all")

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
count,320772.000000,320749,243891.000000,229554.000000,244971.000000,200886.000000,259922.000000,255510.000000,255463.000000,320772.000000
unique,NaN,320579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,70650800367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,160385.500000,NaN,12.730379,5.129932,16.003484,2.862111,7.075940,2.028624,0.798815,65.861511
std,92599.044612,NaN,17.578747,8.014238,22.327284,12.867578,8.409054,128.269454,50.504428,32.091021
min,0.000000,NaN,0.000000,0.000000,-17.860000,-6.700000,-800.000000,0.000000,0.000000,0.000000
25%,80192.750000,NaN,0.000000,0.000000,1.300000,0.000000,0.700000,0.063500,0.025000,41.907242
50%,160385.500000,NaN,5.000000,1.790000,5.710000,1.500000,4.760000,0.581660,0.229000,75.670259
75%,240578.250000,NaN,20.000000,7.140000,24.000000,3.600000,10.000000,1.374140,0.541000,94.145336


In [8]:
product_df["fat_100g"] = product_df["fat_100g"].fillna(product_df["saturated-fat_100g"])
product_df = product_df.fillna(0)

### Valeurs négatives

In [9]:
columns_to_check = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "autre",
]

# 1ere étape : comparaison
negative_matrix_mask = product_df[columns_to_check] < 0

# 2e étape : "agrégation" par col -> 1 seule valeur par ligne
negative_mask = negative_matrix_mask.any(axis=1)
# .any()
# .sum() puis comparer à 100
# donne un masque

# 3e étape : filtrer avec le masque
negative_df = product_df[negative_mask]
# product_df.where(mask).dropna()

# 4e étape : compter le nombre de lignes
len(negative_df)

11

In [10]:
negative_mask.value_counts()

False    320761
True         11
Name: count, dtype: int64

### Valeurs supérieures à 100

In [11]:
columns_to_check = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "autre",
]
tol = 2

# 1ere étape : comparaison
more_than_100_matrix_mask = product_df[columns_to_check] > 100 + tol

# 2e étape : "agrégation" par col -> 1 seule valeur par ligne
more_than_100_mask = more_than_100_matrix_mask.any(axis=1)
# .any()
# .sum() puis comparer à 100
# donne un masque

# 3e étape : filtrer avec le masque
more_than_100_df = product_df[more_than_100_mask]

# 4e étape : compter le nombre de lignes
len(more_than_100_df)

131

In [12]:
columns_to_check = [
    "fat_100g",
    # "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    # "sodium_100g",
    "autre",
]
tol = 2

# 1ere étape : faire la somme
sum_product_series = product_df[columns_to_check].sum(axis=1)

# 2e étape : comparer la somme à 100
# Opérateur logique OU entre deux masques : |
# Opérateur logique ET entre deux masques : &
# ils ont la même priorité algébrique que les opérateurs de comparaison
# different_than_100_mask = (sum_product_series < 100 - tol) | (sum_product_series > 100 + tol)
less_than_100_mask = sum_product_series < 100 - tol
more_than_100_mask = sum_product_series > 100 + tol
different_than_100_mask = less_than_100_mask | more_than_100_mask

# 3e étape : filtrer avec le masque
different_than_100_df = product_df[different_than_100_mask]

# 4e étape : compter le nombre de lignes
len(different_than_100_df)

115367

In [13]:
less_than_100_mask.value_counts()

False    206162
True     114610
Name: count, dtype: int64

In [14]:
more_than_100_mask.value_counts()

False    320015
True        757
Name: count, dtype: int64

In [15]:
# Créer une nouvelle colonne contenant les valeurs de somme
product_df["sum"] = sum_product_series
product_df[more_than_100_mask].sample(20)

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,sum
7983,7983,11213053264,0.77,0.00,2.31,3.8,4.62,781.53768,307.692000,0.0,793.03768
285852,285852,7503008520417,25.00,25.00,64.00,10.0,6.00,6.00000,2.362205,0.0,111.00000
121707,121707,649241838135,0.00,0.00,0.00,0.0,0.00,107.69600,42.400000,0.0,107.69600
140581,140581,748252207234,61.90,47.62,23.81,9.5,9.52,0.56896,0.224000,0.0,105.29896
108392,108392,94776134953,78.57,0.00,3.57,0.0,32.14,4.35356,1.714000,0.0,118.63356
307610,307610,8410954000285,70.00,70.00,60.00,0.0,9.00,23.00000,9.055118,0.0,162.00000
147660,147660,799137017653,53.57,25.00,60.71,3.6,7.14,1.13284,0.446000,0.0,126.15284
42655,42655,41269408015,32.43,27.03,67.57,2.7,2.70,0.17272,0.068000,0.0,105.57272
142393,142393,758424001737,0.00,0.00,90.00,20.0,0.00,0.00000,0.000000,0.0,110.00000
296210,296210,7616500178243,34.00,20.00,57.00,5.0,35.00,0.14000,0.055118,0.0,131.14000


In [16]:
product_df[less_than_100_mask].head(20)

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,sum
1,1,4530,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000,71.43000
3,3,16087,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372,94.15800
7,7,16124,18.75,4.69,15.62,9.4,14.06,0.13970,0.055,37.28530,95.25500
8,8,16193,37.50,22.50,42.50,7.5,5.00,0.00000,0.000,0.00000,92.50000
12,12,16872,36.67,5.00,3.33,6.7,16.67,1.60782,0.633,29.38918,94.36700
14,14,17497,48.48,9.09,0.00,15.2,30.30,0.57658,0.227,0.00000,94.55658
16,16,18050,60.71,3.57,3.57,10.7,14.29,0.01016,0.004,7.14584,96.42600
17,17,18173,26.67,23.33,16.67,3.3,3.33,0.00000,0.000,26.70000,76.67000
20,20,18265,17.50,7.50,32.50,5.0,7.50,0.28448,0.112,29.60352,92.38800
21,21,18289,33.33,6.67,30.00,6.7,13.33,0.46482,0.183,9.32218,93.14700


### Saturated-fat et fat

In [17]:
fat_mask = product_df["saturated-fat_100g"] > product_df["fat_100g"]
fat_mask.value_counts()

False    320418
True        354
Name: count, dtype: int64

In [18]:
product_df[fat_mask].to_csv(Path(DATA_DIR, "weird_fat.csv"))

### Salt et sodium

In [19]:
product_df["sodium_to_salt_ratio"] = product_df["sodium_100g"] / product_df["salt_100g"]

In [20]:
mean_ratio = product_df["sodium_to_salt_ratio"].mean()
tol = 0.01

In [21]:
expected_sodium_inf_series = product_df["salt_100g"] * (mean_ratio - tol)
expected_sodium_sup_series = product_df["salt_100g"] * (mean_ratio + tol)

In [22]:
mask = (product_df["sodium_100g"] < expected_sodium_inf_series) | (product_df["sodium_100g"] > expected_sodium_sup_series)
mask.value_counts()

False    320768
True          4
Name: count, dtype: int64

Bonus : reprendre l'étude d'une typologie de valeurs aberrantes avec Dask.

In [23]:
import dask.dataframe as dd

In [24]:
product_ddf = dd.read_csv(product_file_path, dtype={"code": "object"})

In [26]:
product_ddf.describe().compute()

,Unnamed: 0,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
count,320772.000000,243891.000000,229554.000000,244971.000000,200886.000000,259922.000000,255510.000000,255463.000000,320772.000000
mean,160385.500000,12.730379,5.129932,16.003484,2.862111,7.075940,2.028624,0.798815,65.861511
std,92599.044612,17.578747,8.014238,22.327284,12.867578,8.409054,128.269454,50.504428,32.091021
min,0.000000,0.000000,0.000000,-17.860000,-6.700000,-800.000000,0.000000,0.000000,0.000000
25%,80192.750000,0.000000,0.000000,1.300000,0.000000,0.700000,0.063500,0.025000,41.907242
50%,160385.500000,5.000000,1.790000,5.710000,1.500000,4.760000,0.581660,0.229000,75.670259
75%,240578.250000,20.000000,7.140000,24.000000,3.600000,10.000000,1.374140,0.541000,94.145336
max,320771.000000,714.290000,550.000000,3520.000000,5380.000000,430.000000,64312.800000,25320.000000,889.380000


In [28]:
negative_matrix_mask = product_ddf[columns_to_check] < 0
negative_mask = negative_matrix_mask.any(axis=1)
product_ddf[negative_mask].compute()

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
8582,8582,11213420608,0.00,0.00,-1.20,1.2,2.41,0.38354,0.151000,97.055460
18209,18209,21130493432,0.80,0.00,-0.80,0.8,0.80,0.87376,0.344000,97.182240
23784,23784,28400231053,33.33,13.33,0.00,-6.7,NaN,6.43382,2.533000,51.073180
33781,33781,36800416727,46.43,8.93,3.57,3.6,-3.57,0.99822,0.393000,39.648780
115310,115310,4029816,0.00,NaN,NaN,NaN,-500.00,25.40000,10.000000,564.600000
117739,117739,608866999263,3.57,0.00,-3.57,3.6,7.14,0.95250,0.375000,87.932500
146284,146284,789280259062,13.33,3.33,-6.67,6.7,NaN,2.03200,0.800000,80.478000
150858,150858,813922021028,6.25,1.25,-6.25,1.2,1.25,1.19380,0.470000,94.636200
164030,164030,856336001538,21.43,3.57,-17.86,17.9,17.86,1.93294,0.761000,54.406060
169119,169119,875208001230,0.00,NaN,0.00,NaN,-800.00,7.62000,3.000000,889.380000
